In [1]:
import yaml

import mecadoi.article
import mecadoi.crossref.api
import mecadoi.crossref.peer_review
import mecadoi.model

def load_articles(path):
    return parse_articles(load_articles_dict(path))

def load_articles_dict(path):
    with open(path) as f:
        return yaml.safe_load(f)

def parse_articles(articles_dict):
    return [
        mecadoi.article.Article(
            doi=article["doi"],
            title=article["title"],
            review_process=[
                mecadoi.article.RevisionRound(
                    reviews=[
                        mecadoi.article.Review(
                            authors=[
                                mecadoi.model.Author(
                                    given_name=author["given_name"],
                                    surname=author["surname"],
                                    orcid=mecadoi.model.Orcid(
                                        id=author["orcid"]["id"],
                                        is_authenticated=author["orcid"]["is_authenticated"],
                                    ) if author["orcid"] else None,
                                    is_corresponding_author=author["is_corresponding_author"],
                                    institutions=[mecadoi.model.Institution(name=author["affiliation"])],
                                )
                                for author in review["authors"]
                            ],
                            text=review["text"],
                            doi=review["doi"],
                            publication_date=review["publication_date"],
                        )
                        for review in revision_round["reviews"]
                    ],
                    author_reply=mecadoi.article.AuthorReply(
                        authors=[
                            mecadoi.model.Author(
                                given_name=author["given_name"],
                                surname=author["surname"],
                                orcid=mecadoi.model.Orcid(
                                    id=author["orcid"]["id"],
                                    is_authenticated=author["orcid"]["is_authenticated"],
                                ) if author["orcid"] else None,
                                is_corresponding_author=author["is_corresponding_author"],
                                institutions=(
                                    [mecadoi.model.Institution(name=author["affiliation"])]
                                    if author.get("affiliation", None)
                                    else [
                                        mecadoi.model.Institution(
                                            name=institution["name"],
                                            department=institution.get("department", None),
                                            city=institution.get("city", None),
                                            country=institution.get("country", None),
                                        )
                                        for institution in author["institutions"]
                                    ]
                                    if author.get("institutions", None)
                                    else None
                                ),
                            )
                            for author in revision_round["author_reply"]["authors"]
                        ],
                        text=revision_round["author_reply"]["text"],
                        doi=revision_round["author_reply"]["doi"],
                        publication_date=revision_round["author_reply"]["publication_date"],
                    )
                    if revision_round["author_reply"]
                    else None,
                )
                for revision_round in article["review_process"]
            ]
        )
        for article in articles_dict
    ]

def save_deposition_xml(path, deposition_xml):
    with open(f"/Users/eidens/Projects/mecadoi-archives/legacy/RC/{path.stem}.xml", "w") as f:
        f.write(deposition_xml)

def update(path):
    articles = load_articles(path)
    deposition_xml = mecadoi.crossref.peer_review.generate_peer_review_deposition(articles)
    save_deposition_xml(path, deposition_xml)
    return mecadoi.crossref.api.deposit(deposition_xml)

In [2]:
import pathlib
deposited_articles = [pathlib.Path("/Users/eidens/Projects/mecadoi-archives/batch/deposited/8ce1891c-46ec-41fe-b33d-9f32b6b7129b.yml")]
deposited_articles

[PosixPath('/Users/eidens/Projects/mecadoi-archives/batch/deposited/8ce1891c-46ec-41fe-b33d-9f32b6b7129b.yml')]

In [3]:
articles = [article for path in deposited_articles for article in load_articles(path)]
len(articles)

1

In [4]:
articles[0].doi

'10.1101/055574'

In [5]:
articles[0].doi = "10.1101/2020.04.22.055574"

In [6]:
deposition_xml = mecadoi.crossref.peer_review.generate_peer_review_deposition(articles)
with open(f"update.xml", "w") as f:
        f.write(deposition_xml)

In [11]:
mecadoi.crossref.api.CROSSREF_DEPOSITION_URL

'https://doi.crossref.org/servlet/deposit'

In [8]:
deposition_xml

'<doi_batch xmlns="http://www.crossref.org/schema/5.3.1" xmlns:rel="http://www.crossref.org/relations.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="5.3.1" xsi:schemaLocation="http://www.crossref.org/schema/5.3.1 http://www.crossref.org/schemas/crossref5.3.1.xsd">\n  <head>\n    <doi_batch_id>rc.1687788814598726000</doi_batch_id>\n    <timestamp>1687788814598726000</timestamp>\n    <depositor>\n      <depositor_name>EMBO</depositor_name>\n      <email_address>eidens@embl.de</email_address>\n    </depositor>\n    <registrant>EMBO</registrant>\n  </head>\n  <body>\n    <peer_review language="en" revision-round="0" type="referee-report" stage="pre-publication">\n      <contributors>\n        <anonymous sequence="first" contributor_role="author"/>\n      </contributors>\n      <titles>\n        <title>Peer Review #1 of FAM111A regulates replication origin activation and cell fitness</title>\n      </titles>\n      <review_date>\n        <month>4</month>\n        <day>1

In [9]:
mecadoi.crossref.api.deposit(deposition_xml)

'\n\n\n\n<html>\n<head><title>SUCCESS</title>\n</head>\n<body>\n<h2>SUCCESS</h2>\n<p>Your batch submission was successfully received.</p>\n</body>\n</html>\n'